# DNNで二値分類をする
keras_tunerを使用して、ハイパーパラメータのチューニングを行う。

### 大まかな処理の流れ
1. データの取り込み
2. データの整形（トレーニングデータとテストデータを作成）
3. モデルの定義
4. チューナーをインスタンス化して、ハイパーパラメータのチューニングする
5. チューニングしたハイパーパラメータでモデルを作成し、トレーニングする
6. 作成したモデルを保存する
7. 保存したモデルをお読み込み、予測したいデータで予測する

## 使用ライブラリのインポート

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.utils import FeatureSpace
import keras_tuner
from tensorflow import keras

## 1. データの取り込み

In [2]:
file_url = "http://storage.googleapis.com/download.tensorflow.org/data/heart.csv"
dataframe = pd.read_csv(file_url)

## 2. データの確認
省略可能

In [3]:
print(dataframe.shape)

(303, 14)


In [ ]:
dataframe.head()

## 3. トレーニングデータと検証データの作成

In [ ]:
# データフレームから検証用としてランダムに20％取り出す
val_dataframe = dataframe.sample(frac=0.2, random_state=1337)
train_dataframe = dataframe.drop(val_dataframe.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_dataframe), len(val_dataframe))
)

In [ ]:
# データフレームからデータセットに変換
def dataframe_to_dataset(dataframe):
    dataframe = dataframe.copy()
    labels = dataframe.pop("target")
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    ds = ds.shuffle(buffer_size=len(dataframe))
    return ds

train_ds = dataframe_to_dataset(train_dataframe)
val_ds = dataframe_to_dataset(val_dataframe)

In [ ]:
# データセットの中身の確認
for x, y in train_ds.take(1):
    print("Input:", x)
    print("Target:", y)

In [ ]:
train_ds = train_ds.batch(32)
val_ds = val_ds.batch(32)

In [ ]:
# 構造化データの前処理とエンコードを行う
feature_space = FeatureSpace(
    features={
        # integerとしてエンコードされたカテゴリー特徴
        "sex": "integer_categorical",
        "cp": "integer_categorical",
        "fbs": "integer_categorical",
        "restecg": "integer_categorical",
        "exang": "integer_categorical",
        "ca": "integer_categorical",
        # stringとしてエンコードされたカテゴリー特徴
        "thal": "string_categorical",
        # 離散化する数値的特徴
        "age": "float_discretized",
        # 正規化する数値的特徴
        "trestbps": "float_normalized",
        "chol": "float_normalized",
        "thalach": "float_normalized",
        "oldpeak": "float_normalized",
        "slope": "float_normalized",
    },
    # 交差する特徴の組み合わせリスト。
    # 特徴は、結合された値を固定長ベクトルにハッシュすることによって「交差」する。
    crosses=[("sex", "age"), ("thal", "ca")],
    # 交差した特徴をハッシュするためのデフォルトのベクトルサイズ
    # デフォルトは32
    crossing_dim=32,
    # "concat"または"dict"。
    # "concat"では、すべての特徴が 1 つのベクトルに連結される。
    # "dict"では、個別にエンコードされた特徴のdictを返す (入力キーと同じキーを使用)。
    output_mode="concat",
)

In [ ]:
train_ds_with_no_labels = train_ds.map(lambda x, _: x)
feature_space.adapt(train_ds_with_no_labels)

In [ ]:
# トレーニングデータセットから１つ取り出して確認
for x, _ in train_ds.take(1):
    preprocessed_x = feature_space(x)
    print("preprocessed_x.shape:", preprocessed_x.shape)
    print("preprocessed_x.dtype:", preprocessed_x.dtype)

In [ ]:
# num_parallel_callsで処理を並列化する。
# tf.data.AUTOTUNEは並列度をランタイムで良い感じに決めてくれる。
preprocessed_train_ds = train_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
# prefetchはGPUが計算している間にBatchデータをCPU側で用意しておく機能
preprocessed_train_ds = preprocessed_train_ds.prefetch(tf.data.AUTOTUNE)

preprocessed_val_ds = val_ds.map(
    lambda x, y: (feature_space(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
preprocessed_val_ds = preprocessed_val_ds.prefetch(tf.data.AUTOTUNE)

print(preprocessed_train_ds)

## 4. モデルを定義

In [ ]:
dict_inputs = feature_space.get_inputs()
encoded_features = feature_space.get_encoded_features()

def build_model(hp):
    model = tf.keras.Sequential()

    # 入力層
    model.add(tf.keras.Input(tensor=encoded_features))

    # 隠れ層
    hp_n_hidden_layers = hp.Int("n_hidden_layers", min_value=1, max_value=10) # max_valueに最適化した最大隠れ層数を指定
    for i in range(hp_n_hidden_layers):
        hp_units = hp.Int("units_%d" % (i + 1), min_value=32, max_value=512, step=32)
        hp_activation = hp.Choice("activation_%d" % (i + 1), ["relu", "tanh"]) # 活性化関数
        model.add(tf.keras.layers.Dense(hp_units, activation=hp_activation))
        model.add(tf.keras.layers.Dropout(hp.Choice(name="dropout_%d" % (i + 1), values=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))) # ドロップアウト率

    # 出力層
    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

    # 最適化アルゴリズム、損失関数、評価関数を指定してコンパイル
    # 最適化アルゴリズムを最適化
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    model.compile(
        # 最適化アルゴリズム
        optimizer=tf.keras.optimizers.Adam(hp_learning_rate),
        # 損失関数
        # 二値分類→binary_crossentropy、多クラス単一ラベル分類→categorical_crossentropy
        # 多クラス多ラベル分類→binary_crossentropy、回帰問題（任意の値）→mse、回帰問題（０～１の値）→mse / binary_crossentropy
        loss="binary_crossentropy",
        # 評価関数
        metrics=["accuracy"],
    )

    return model

## 5. チューナーをインスタンス化してハイパーパラメータのチューニングを実行

In [ ]:
# チューナーのインスタンス化
# RandomSearch、Hyperband、BayesianOptimization、Sklearnがある
tuner = keras_tuner.BayesianOptimization(
    hypermodel=build_model,
    objective="val_accuracy", # 最適化する目標の名前：accuracy, loss, val_accuracy, val_loss
    max_trials=50, # チューニングする試行数
    executions_per_trial=2, # 各トライアルに構築して適合させる必要があるモデルの数
    overwrite=True,
    directory="./",
    project_name="dnn_keras_tuner",
)

In [ ]:
# 検証損失が特定の値に達した時に、トレーニングを早期に停止するためのコールバック関数
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
cur_dir_path = %pwd
tuner.search(
    preprocessed_train_ds, # トレーニングデータセット
    epochs=50, # 学習の繰り返す数
    validation_data=preprocessed_val_ds, # 検証データセット
    callbacks=[stop_early], # 早期停止する場合のコールバック関数
)
best_model = tuner.get_best_models()[0]

## 6. モデルをトレーニング

In [ ]:
# 最適なハイパーパラメータ
best_hp = tuner.get_best_hyperparameters()[0]
# 最適なハイパーパラメータでモデルの作成
model = tuner.hypermodel.build(best_hp)
# モデルのトレーニング
model.fit(preprocessed_train_ds, epochs=50, validation_data=preprocessed_val_ds)

## 7. モデルの保存と概要

In [ ]:
# モデルの保存
model.save("model")
# モデルの概要
model.summary()

In [ ]:
# チューニングの該当
tuner.results_summary()

In [ ]:
# モデルの読み込み
model = tf.keras.models.load_model('model')

sample = {
    "age": 60,
    "sex": 1,
    "cp": 1,
    "trestbps": 145,
    "chol": 233,
    "fbs": 1,
    "restecg": 2,
    "thalach": 150,
    "exang": 0,
    "oldpeak": 2.3,
    "slope": 3,
    "ca": 0,
    "thal": "fixed"
}
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

# 予測
output = model.predict(feature_space(input_dict))

In [ ]:
# 離散化を行う関数
def discretize(proba):
    threshold = np.array([0.5]) # 0か1かを分ける閾値を0.5に設定
    discretized = (proba >= threshold).astype(int) # 閾値未満で0、以上で1に変換
    return discretized

In [ ]:
# 予測結果
print(np.array(discretize(output)))